In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
from sklearn import preprocessing

In [6]:
#LOOP SE SI HA UN NUMERO DI BRANI PARI A MULTIPLO DI 20(Semplicemente controllare il codice di ritorno della richiesta, così tolgo anche il conto) 

def requestToken():
    #Setto lo scope per la richiesta
    scope = "user-library-read"
    #autorizzo l'applicativo e prendo il token
    sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))
    return sp

def extract_song_info(track,idx,i):
    #Si crea il dizionario e si aggiunge alla lista
        artist=track['artists'][0]['name']
        song_title=track['name']
        song_uri=track['uri']
        data={
            'Artist': artist,
            'Song': song_title,
            'uri': song_uri
        }
        return data



def build_dict(sp):
    #Avvio l'offset
    i=0
    #preparo la lista di dizionari per costruire il dataframe(test su Stack Overflow dicono sia il metodo più veloce)
    row_list= []
    #Il metodo usato permette di estrarre solo 20 tracce alla volta, quindi si ripeterà
    #Finchè non prende meno di 20 tracce
    count=19
    while count >=19:
        #Salva le 20 tracce in una variabile con tutti i json
        results = sp.current_user_saved_tracks(limit=20,offset=i*20)
        #Si scorre la lista di json
        for idx, item in enumerate(results['items']):
            track = item['track']
            
            #Stampo per avere un'idea di dove si trovi l'estrazione
            print(idx+(20*i), track['artists'][0]['name'], " – ", track['name'], "- ", track['uri'])
            
            data=extract_song_info(track,idx,i)
            row_list.append(data)
            
            #Si controlla qual è l'ultimo indice
            count=idx
        i+=1
    return row_list

def extract_song_df():
    sp=requestToken()
    row_list=build_dict(sp)
    df=pd.DataFrame(row_list, columns=['Artist','Song','uri'])
    return df

    

In [7]:
df=extract_song_df()
df

na Perfecta Excusa -  spotify:track:2mKdCml4XxngXh8RRA5mSP
4077 Modena City Ramblers  –  Clan Banlieu -  spotify:track:0QFd8mVZLEPBClhijcRTN2
4078 Modena City Ramblers  –  Mia Dolce Rivoluzionaria -  spotify:track:02pYXGf22z0PVgbAFbBqd4
4079 Modena City Ramblers  –  I Cento Passi -  spotify:track:3AkYGH0Zm8z3l8Mla4bLM3
4080 Modena City Ramblers  –  Bella Ciao Partisan -  spotify:track:4LVilmd93YwMCnMpBDHXBq
4081 Modena City Ramblers  –  La Banda Del Sogno Interrotto -  spotify:track:6YpBkUHi9t64crAN5VAyX5
4082 Modena City Ramblers  –  Mondina's Bella Ciao -  spotify:track:4QH2o5p991VV1VyhYOVVtD
4083 Modena City Ramblers  –  Roisin The Bow -  spotify:track:5cEPRwyw8jEfIYmAY2ll81
4084 Modena City Ramblers  –  Onda libera -  spotify:track:7DAGveRAznPYsOKr5dfj5l
4085 Modena City Ramblers  –  Libera terra -  spotify:track:0y3T1Dkryacbr4zLtX6wZz
4086 Modena City Ramblers  –  Valzer chiuso in soffitta -  spotify:track:0dcE35ESgiDlJsQ95zcA0v
4087 Modena City Ramblers  –  Figli del vento -  spo

,Artist,Song,uri
0,Mac DeMarco,Another One,spotify:track:3pXAa69soxZ98bQ1gr18HO
1,alt-J,Breezeblocks,spotify:track:3n69hLUdIsSa1WlRmjMZlW
2,Francesco De Gregori,Rimmel,spotify:track:515XcapFOMtOOiGU31UqNp
3,Vinicio Capossela,Marajà - 2018 Remaster,spotify:track:70EXIyETxgloAlVkRbO2H9
4,The Books,The Lemon of Pink I,spotify:track:6WkwB5tjvgw8zjdKSOfl0S
...,...,...,...
4313,Caparezza,Messa In Moto,spotify:track:6cg4SNSGiqYRA34opvo2G2
4314,Caparezza,Non Siete Stato Voi,spotify:track:69kLZ80Yv9v35H6PLfmnjc
4315,Caparezza,La Ghigliottina,spotify:track:3uisLZ2RNXGAF6rkGfquXF
4316,Caparezza,Ti Sorrido Mentre Affogo,spotify:track:1oszMEEhMfia4uecZwZPDk


In [18]:
#Adesso si prenderanno tutte le feature di interesse per ogni canzone
#Scorro il dataframe con un ciclo per necessità di alcuni comportamenti specifici, il dataframe non è molto grande comunque

def find_feature(uri):
    sp=requestToken()
    return sp.audio_features(tracks=uri)

def create_dictionary(feature):
    if feature is None or len(feature) == 0 or feature[0] is None:
        #raise ValueError("Qualcosa è andato storto nella richiesta della feature")
        return None

    return {
        'uri': feature[0]['uri'],
        'Acousticness': feature[0]['acousticness'],
        'Danceability': feature[0]['danceability'],
        'Liveness': feature[0]['liveness'],
        'Loudness':feature[0]['loudness'],
        'Speechiness':feature[0]['speechiness'],
    }

def add_to_list(dictionary, feature_list):
    if dictionary is not None:
        feature_list.append(dictionary)

#TEST VARI
#feature_list=[]
#feature=find_feature('spotify:track:0QFd8mVZLEPBClhijcRTN2')
#print(create_dictionary(feature))
#add_to_list(create_dictionary(feature),feature_list)
#feature_list

def create_dict_list(df):
    feature_list=[]
    for i in range(len(df)):
        row=df.iloc[i]
        add_to_list(create_dictionary(find_feature(row['uri'])),feature_list)
        print(i,row['Song'])
    return feature_list


def extract_features_df():
    
    d_list=create_dict_list(df)
    df2=pd.DataFrame(d_list, columns=['uri','Acousticness','Danceability','Liveness','Loudness','Speechiness'])
    return df2

In [20]:
df2=extract_features_df()
df2

etetlen
3488 Jó
3489 Hetedhét székely
3490 A jó ég megáld
3491 Hozz egy firkin sört
3492 Búcsúpohár
3493 If I Could Be the Pope
3494 Kind and Fine
3495 Molly Malone
3496 High and Low
3497 The Child of Fire
3498 Loch Lomond
3499 Dirty Julie
3500 Donegal Danny
3501 Parasite's Lust
3502 I Am Who I Am
3503 Finger in the Pie
3504 My Love John
3505 Jumpin' Lovers
3506 Nincompoop
3507 Il secondo secondo me
3508 Il naufrago del lusitalia
3509 In Vento Intro
3510 In Vento
3511 Italia Italia
3512 Test di Ingresso di Medicina
3513 Jackie il Melo Drammatico
3514 Bagatelle
3515 Ilaria
3516 Cancelleria
3517 Castagne Genge
3518 Sudowoodo
3519 Rodger
3520 Django
3521 Il Sonno ed il Furto
3522 La Strategia della Tenzone
3523 Le Gentil
3524 Me Want Marò Back
3525 Dobrij Abend
3526 Zakoni Podlosti
3527 Dawai, Dawai
3528 Zaporozhez
3529 Dope Shit (Radio Edit)
3530 Barabani
3531 Dope Shit (Original Version)
3532 Dope Shit (Balkanizer Remix)
3533 Dope Shit (Afrit Remix)
3534 Dope Shit (Putnik Remix)
3535 Do

,uri,Acousticness,Danceability,Liveness,Loudness,Speechiness
0,spotify:track:3pXAa69soxZ98bQ1gr18HO,0.821000,0.694,0.106,-10.325,0.0276
1,spotify:track:3n69hLUdIsSa1WlRmjMZlW,0.096000,0.616,0.205,-7.298,0.0344
2,spotify:track:515XcapFOMtOOiGU31UqNp,0.620000,0.500,0.106,-14.525,0.0544
3,spotify:track:70EXIyETxgloAlVkRbO2H9,0.172000,0.486,0.059,-6.453,0.0913
4,spotify:track:6WkwB5tjvgw8zjdKSOfl0S,0.887000,0.626,0.178,-13.197,0.0529
...,...,...,...,...,...,...
4312,spotify:track:6cg4SNSGiqYRA34opvo2G2,0.001350,0.573,0.235,-4.496,0.2090
4313,spotify:track:69kLZ80Yv9v35H6PLfmnjc,0.020400,0.409,0.143,-5.058,0.2970
4314,spotify:track:3uisLZ2RNXGAF6rkGfquXF,0.000676,0.685,0.132,-5.779,0.0535
4315,spotify:track:1oszMEEhMfia4uecZwZPDk,0.001030,0.750,0.441,-4.900,0.0787


In [34]:
df3=pd.merge(df,df2,on='uri')
#POSSO USARE QUESTI PRINT PER TARARE L'ACCUMULATORE (DA AUTOMATIZZARE)
print(df3['Loudness'].max())
print(df3['Loudness'].min())

-0.721
-37.521


In [29]:
#Normalizzo la Loudness\n
def normalize_loudness_df(df3):
    loudness = df3[['Loudness']].values
    min_max_scaler = preprocessing.MinMaxScaler()
    loudness_scaled = min_max_scaler.fit_transform(loudness)
    df3=df3.assign(Loudness=loudness_scaled)
    return df3


In [30]:
df3= normalize_loudness_df(df3)
    
df3.to_csv(r'C:/Users/capiz/Desktop/UNIVERSITA/Tap/Progetti/final_dataframe.csv')

df3

,Artist,Song,uri,Acousticness,Danceability,Liveness,Loudness,Speechiness
0,Mac DeMarco,Another One,spotify:track:3pXAa69soxZ98bQ1gr18HO,0.821000,0.694,0.106,0.739022,0.0276
1,alt-J,Breezeblocks,spotify:track:3n69hLUdIsSa1WlRmjMZlW,0.096000,0.616,0.205,0.821277,0.0344
2,Francesco De Gregori,Rimmel,spotify:track:515XcapFOMtOOiGU31UqNp,0.620000,0.500,0.106,0.624891,0.0544
3,Vinicio Capossela,Marajà - 2018 Remaster,spotify:track:70EXIyETxgloAlVkRbO2H9,0.172000,0.486,0.059,0.844239,0.0913
4,The Books,The Lemon of Pink I,spotify:track:6WkwB5tjvgw8zjdKSOfl0S,0.887000,0.626,0.178,0.660978,0.0529
...,...,...,...,...,...,...,...,...
4312,Caparezza,Messa In Moto,spotify:track:6cg4SNSGiqYRA34opvo2G2,0.001350,0.573,0.235,0.897418,0.2090
4313,Caparezza,Non Siete Stato Voi,spotify:track:69kLZ80Yv9v35H6PLfmnjc,0.020400,0.409,0.143,0.882147,0.2970
4314,Caparezza,La Ghigliottina,spotify:track:3uisLZ2RNXGAF6rkGfquXF,0.000676,0.685,0.132,0.862554,0.0535
4315,Caparezza,Ti Sorrido Mentre Affogo,spotify:track:1oszMEEhMfia4uecZwZPDk,0.001030,0.750,0.441,0.886440,0.0787
